# Python Script Code
## Import relevant packages/libraries

In [1]:
import os
import re
import csv

In [2]:
def Second_counter(time):
    return int(time[0]) * 36000 + int(time[1]) * 3600 + int(time[2]) * 600 + int(time[3]) * 60 + int(time[4]) * 10 + int(time[5])

## Function to get data from Place log file

In [3]:
def dataPlace(placeFile):
   
    with open(placeFile,"r") as pf:
        lines = pf.readlines()
    for line in lines:
        RD = re.search("Row Direction", line)
        CU = re.search("Core Utilization", line)
        NR = re.search("Number Of Rows", line)
        CW = re.search("Core Width", line)
        CH = re.search("Core Height", line)
        AR = re.search("Aspect Ratio =", line)
        DB = re.search("Double Back", line)
        FFR = re.search("Flip First Row", line)
        SFFR = re.search("Start From First Row", line)
        ST = re.search(r"Starting ICC placement run at: \S+ \d+ \d+ (\d+):(\d+):(\d+)",line)
        ET = re.search(r"ICC placement run @: \S+ \d+ \d+ (\d+):(\d+):(\d+)",line)
        
        if RD:
            RowDirection = re.match(r"\sRow Direction = (\w+)", line).group(1)
        if CU:
            CoreUtils = re.match(r"\sCore Utilization = (\S+)", line).group(1)
        if NR:
            NoRows = re.match(r"\sNumber Of Rows = (\S+)", line).group(1)
        if CW:
            CoreWidth = re.match(r"\sCore Width = (\S+)", line).group(1)
        if CH:
            CoreHeight = re.match(r"\sCore Height = (\S+)", line).group(1)
        if AR:
            AspectRatio = re.match(r"\sAspect Ratio = (\S+)", line).group(1)
        if DB:
            DoubleBack = re.match(r"\sDouble Back (\w+)", line).group(1)
        if FFR:
            FlipFirstRow = re.match(r"\sFlip First Row = (\w+)", line).group(1)
        if SFFR:
            StartFromFirstRow = re.match(r"\sStart From First Row = (\w+)", line).group(1)
        if ST:
            StartTime = ST.group(1)+ST.group(2)+ST.group(3)
        if ET:
            EndTime = ET.group(1)+ET.group(2)+ET.group(3)            
    
    RunTimePlace = Second_counter(EndTime)-Second_counter(StartTime)    
    
    return RowDirection,float(CoreUtils),int(NoRows),float(CoreWidth),float(CoreHeight),float(AspectRatio),DoubleBack,FlipFirstRow,StartFromFirstRow,RunTimePlace

## Function to get data from Route log file

In [4]:
def dataRoute(routeFile):
    with open(routeFile,"r") as rf:
        lines = rf.readlines()
    Violations=0
    RoutedContacts=0
    TotalNets=0
    flagAV=0
    LayerMX = dict(M1=0,M2=0,M3=0,M4=0,M5=0,M6=0,M7=0)
    LayerLX = dict(L1=0,L2=0,L3=0,L4=0)
    LayerBX = dict(BA=0,BB=0,BD=0,BE=0,BG=0)
    LayerFX = dict(FA=0,FB=0)
    Layer3X = dict(U3T=0,U3A=0)
    LayerLB = dict(LB=0)
    for line in lines:
        V = re.search("VIOLATIONS", line)
        RC = re.search("Routed Contacts", line)
        LMX = re.search('Layer\s+(M[1-7])\s+:\s+(\d+)',line)
        LLX = re.search('Layer\s+(L[1-7])\s+:\s+(\d+)',line)
        LBX = re.search('Layer\s+(B[1-7|A-Z])\s+:\s+(\d+)',line)
        LFX = re.search('Layer\s+(F[A-Z])\s+:\s+(\d+)',line)
        L3X = re.search('Layer\s+(3[A-Z])\s+:\s+(\d+)',line)
        LLB = re.search('Layer\s+(LB)\s+:\s+(\d+)',line)
        TN = re.search("Total number of nets", line)
        AV = re.search("Total number of antenna violations = (\d+)", line)
        ST = re.search(r"route and Metalfill\) at: \S+ \d+ \d+ (\d+):(\d+):(\d+)",line)   
        ET = re.search(r"finished zroute_final run @: \S+ \d+ \d+ (\d+):(\d+):(\d+)",line)
        
        if V:
            Violations = re.match(r"\s@@@@@@@ TOTAL VIOLATIONS =	(\S+)", line).group(1)
        if RC:
            RoutedContacts = re.match(r"Total Number of Routed Contacts =       (\S+)", line).group(1)
        if TN:
            TotalNets = re.match(r"Total number of nets = (\d+)", line).group(1)
        if LMX:
            LayerMX[LMX.group(1)] = int(LMX.group(2))
        if LLX:
            LayerLX[LLX.group(1)] = int(LLX.group(2))
        if LBX:
            LayerBX[LBX.group(1)] = int(LBX.group(2))
        if LFX:
            LayerFX[LFX.group(1)] = int(LFX.group(2))
        if L3X:
            Layer3X[L3X.group(1)] = int(L3X.group(2))
        if LLB:
            LayerLB[LLB.group(1)] = int(LLB.group(2))
        if AV:
            flagAV=1
            AntennaViolations = re.match(r"Total number of antenna violations = (\d+)", line).group(1)
        if ST:
            StartTime = ST.group(1)+ST.group(2)+ST.group(3)
        if ET:
            EndTime = ET.group(1)+ET.group(2)+ET.group(3)
            
    if flagAV==0:
        AntennaViolations = 0
        
    RunTimeRoute = Second_counter(EndTime)-Second_counter(StartTime)
    
    return int(Violations),int(RoutedContacts),int(TotalNets),LayerMX,LayerLX,LayerBX,LayerFX,Layer3X,LayerLB,int(AntennaViolations),RunTimeRoute

## Append data from both functions to a list
 - if other file of same metal stack, date and 40LP is found, add data to same row

In [6]:
dictResultsList = []
stackCounter = 0

for root, dirs, files in os.walk(r"C:\Users\Asus\Desktop\My Documents\NUS\IA\40LP test\40LP", topdown=False):
    for name in files:
        filename = (os.path.join(root, name))
        flagOtherFile = 0
        if 'place' in name:
            type40LP = str(filename).split('\\')[9]
            metalStack = str(filename).split('\\')[10]
            date = str(filename).split('\\')[11][10:16]  
            for length in range(len(dictResultsList)):
                if metalStack == dictResultsList[length][0] and date == dictResultsList[length][1] and type40LP == dictResultsList[length][2]:
                    k = length
                    flagOtherFile = 1
            if flagOtherFile == 0:
                dictResultsList.append([])
                k = stackCounter
                dictResultsList[k].append(metalStack)
                dictResultsList[k].append(date)
                dictResultsList[k].append(type40LP)
                stackCounter = stackCounter + 1
            
            dictResultsList[k].append(dataPlace(filename))
                
        if 'route' in name:
            metalStack = str(filename).split('\\')[10]
            date = str(filename).split('\\')[11][13:19]
            type40LP = str(filename).split('\\')[9]
            for length in range(len(dictResultsList)):
                if metalStack == dictResultsList[length][0] and date == dictResultsList[length][1] and type40LP == dictResultsList[length][2]:
                    k = length
                    flagOtherFile = 1
            if flagOtherFile == 0:
                dictResultsList.append([])
                k = stackCounter
                dictResultsList[k].append(metalStack)
                dictResultsList[k].append(date)
                dictResultsList[k].append(type40LP)
                stackCounter = stackCounter + 1
                
            dictResultsList[k].append(dataRoute(filename))                

## Write results to csv file (PNRdatabase3)

In [8]:
with open('PNRdatabase3.csv', mode='a') as csv_file:
    fieldnames = ['Violations','AntennaViolations','RunTimeRoute','RunTimePlace','LayerM1','LayerM2','LayerM3','LayerM4','LayerM5','LayerM6','LayerM7','LayerL1','LayerL2','LayerL3','LayerL4','LayerBA','LayerBB','LayerBD','LayerBE','LayerBG','LayerFA','LayerFB','LayerU3T','LayerU3A','LayerLB', 'ViaCounts', 'NetCount','MetalStack','Date','Type40LP', 'RowDirection', 'RowCount', 'CoreUtil', 'CoreWidth', 'CoreHeight', 'AspectRatio', 'DoubleBack', 'FlipFirstRow', 'StartfromFirstRow']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    if os.stat('PNRdatabase3.csv').st_size == 0:
        writer.writeheader()

    for i in range(len(dictResultsList)):
        dictResults = dict(Violations=dictResultsList[i][4][0],AntennaViolations=dictResultsList[i][4][9],RunTimeRoute=dictResultsList[i][4][10],RunTimePlace=dictResultsList[i][3][9],LayerM1=dictResultsList[i][4][3].get('M1'),LayerM2=dictResultsList[i][4][3].get('M2'),LayerM3=dictResultsList[i][4][3].get('M3'),LayerM4=dictResultsList[i][4][3].get('M4'),LayerM5=dictResultsList[i][4][3].get('M5'),LayerM6=dictResultsList[i][4][3].get('M6'),LayerM7=dictResultsList[i][4][3].get('M7'),LayerL1=dictResultsList[i][4][4].get('L1'),LayerL2=dictResultsList[i][4][4].get('L2'),LayerL3=dictResultsList[i][4][4].get('L3'),LayerL4=dictResultsList[i][4][4].get('L4'),LayerBA=dictResultsList[i][4][5].get('BA'),LayerBB=dictResultsList[i][4][5].get('BB'),LayerBD=dictResultsList[i][4][5].get('BD'),LayerBE=dictResultsList[i][4][5].get('BE'),LayerBG=dictResultsList[i][4][5].get('BG'),LayerFA=dictResultsList[i][4][6].get('FA'),LayerFB=dictResultsList[i][4][6].get('FB'),LayerU3T=dictResultsList[i][4][7].get('U3T'),LayerU3A=dictResultsList[i][4][7].get('U3A'),LayerLB=dictResultsList[i][4][8].get('LB'), ViaCounts=dictResultsList[i][4][1],NetCount= dictResultsList[i][4][2], MetalStack= dictResultsList[i][0],Date=dictResultsList[i][1],Type40LP=dictResultsList[i][2],RowDirection= dictResultsList[i][3][0], RowCount= dictResultsList[i][3][2],CoreUtil= dictResultsList[i][3][1],CoreWidth= dictResultsList[i][3][3], CoreHeight= dictResultsList[i][3][4], AspectRatio= dictResultsList[i][3][5], DoubleBack= dictResultsList[i][3][6], FlipFirstRow= dictResultsList[i][3][7], StartfromFirstRow= dictResultsList[i][3][8])
        writer.writerow(dictResults)

## Delete duplicate rows

In [9]:
import fileinput
seen = set() # set for fast O(1) amortized lookup
for line in fileinput.FileInput('PNRdatabase3.csv', inplace=1):
    if line in seen:
        continue # skip duplicate
    seen.add(line)
    print(line), # standard output is now redirected to the file